### Making the function of `np.linalg` and analyzing
> I'll study about the Theorems that i studied in 'Linear Algebra' class 

In [2]:
import numpy as np

#### Random Matrix

In [ ]:
# Making the random matrix within 1 and 2 dimension
m, n = map(int, input().split())

arr = np.random.randint(-100, 101, (m, n), dtype='int16')
print(arr)


[[ 34]
 [-21]
 [ 28]
 [ 75]]


### Function checking the matrix whether it is a triangular or symmetric or diagoanl
> Only based on it's definition, not on other theorems like determinant

In [28]:
# Upper triangular: for all 0 <= i, j < n, i > j ; a[i][j] == 0

def ut(arr):
    if arr.shape == (1,):
        return True
    flag = False        # flag를 통해 행렬 내부에 삼각행렬의 정의에 위반되는 원소가 존재하는지 찾는다.
    n, _ = arr.shape    # Upper triangular는 n x n 행렬에서 정의된다.

    for j in range(n):
        if flag:
            break       # 반복문에서 flag의 값이 True로 변했다면 더이상 조건문을 돌릴 필요가 없다.
        for i in range(j + 1, n):
            if arr[i][j] != 0:      # arr의 i행 j열 원소 ; (i, j)th entry of a matrix, arr
                flag = True         # Upper triangular의 정의 조건에 위배될 때 flag = True
    if flag:
        return False    
    return True

arr = np.array([0])
print(ut(arr))
            


True


Lower Triangular의 함수도 같은 방식으로 선언된다.

In [ ]:
# Symmetric: for all 0 <= i, j < n ; a[i][j] == a[j][i] ; which is a transpose of given matrix

def sym(arr):
    if arr.shape == (1,):
        return True
    n, _ = arr.shape
    for i in range(n):
        for j in range(i + 1, n):       # 모든 (i, j)를 둘러볼 필요가 없다.
            if arr[i][j] != arr[j][i]:
                return False            # flag보다 더 효율적인 방식이다.
    return True                         # for문이 중간에 끊기지 않음 = 주어진 행렬이 symmetric이다.

arr = np.array([[0, 0], [0, 1]])
print(sym(arr))


True


In [27]:
# test

arr = np.zeros(1)
print(arr)
print(arr.shape)

[0.]
(1,)
